In [1]:
import os
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from lxml import etree

# Libraries for multiprocessing
import multiprocessing as mp
from multiprocessing import Pool

In [2]:
# Set corpus to the folder of files you want to use
corpus = '/home/ec2-user/SageMaker/data/us_healthcare/'

# Read in files
input_files = os.listdir(corpus)

print("Loaded", len(input_files), "documents.")

Loaded 48310 documents.


In [3]:
# Modify output_file to desired save name
output_file = 'intermediate/healthcare.csv'

In [4]:
# Check core count
num_cores = mp.cpu_count()
print(num_cores)

8


In [5]:
# Function to strip html tags from text portion
def strip_html_tags(text):
    stripped = BeautifulSoup(text).get_text().replace('\n', ' ').replace('\\', '').strip()
    return stripped

In [6]:
# Retrieve metadata from XML document
def getxmlcontent(corpus, file, strip_html=True):
    try:
        tree = etree.parse(corpus + file)
        root = tree.getroot()

        if root.find('.//GOID') is not None:
            goid = root.find('.//GOID').text
        else:
            goid = None

        if root.find('.//Title') is not None:
            title = root.find('.//Title').text
        else:
            title = None

        if root.find('.//NumericDate') is not None:
            date = root.find('.//NumericDate').text
        else:
            date = None
            
        if root.find('.//PublisherName') is not None:
            publisher = root.find('.//PublisherName').text
        else:
            publisher = None

        if root.find('.//FullText') is not None:
            text = root.find('.//FullText').text

        elif root.find('.//HiddenText') is not None:
            text = root.find('.//HiddenText').text

        elif root.find('.//Text') is not None:
            text = root.find('.//Text').text

        else:
            text = None

        # Strip html from text portion
        if text is not None and strip_html == True:
            text = strip_html_tags(text)
    
    except Exception as e:
        print(f"Error while parsing file {file}: {e}")
    
    return goid, title, date, publisher, text

In [7]:
# Function to make lists out of parsed data--on single document scale for multiprocessing
def make_lists(file):
    
    goid, title, date, publisher, text = getxmlcontent(corpus, file, strip_html=True)
    
    if isinstance(text, str):
        # 1. Lowercase
        text = text.lower() # 1. Lowercase
        
        # 2. Remove punctuation
        translator = str.maketrans(string.punctuation, " " * len(string.punctuation))
        text = text.translate(translator)
        
        # 3. Remove stopwords
        # Define a set of common English stop words
        stop_words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",
                      "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
                      'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',
                      'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
                      'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
                      'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
                      'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above',
                      'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then',
                      'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
                      'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
                      'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
                      "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
                      "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
                      'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}
        
        words = text.split()
        filtered_words = [word for word in words if word not in stop_words]
        text = ' '.join(filtered_words)
        
        # 4. Remove numbers
        text = re.sub(r'\d+', '', text)
        
        # 5. Selection Stemming
        ## Healthcare
        text = re.sub("health-care", "healthcare", text)
        text = re.sub("health care", "healthcare", text)
        text = re.sub("health insuranc ", "healthcare", text)
        ## Obamacare
        text = re.sub(r"\b(obama-care|obama\s+care)\b", "obamacare", text)
        ## Replace terms related to race with "racialization"
        text = re.sub(r"\b(racism|racial|racist|racists|racialized|racializes|racialize|racializing)\b", "racialization", text, flags=re.IGNORECASE)
        ## Replace terms related to African Americans with "blck"
        text = re.sub(r"\b(african\s+american|african\s+americans|african-american|blacks|black\s+men|black\s+man|black\s+woman|black\s+women|black\s+people)\b", "blck", text, flags=re.IGNORECASE)
        ## Replace terms related to Hispanic/Latino with "latinxdad"
        text = re.sub(r"\b(hispanic|hispanic\s+americans|hispanic\s+american|latinos|latinas|latino|latina|latin\s+americans|latin\s+american|latinx|central\s+americans|central\s+american|mexicans|mexican|mexican\s+americans|mexican\s+american|cubans|cuban|cuban\s+americans|cuban\s+american|salvadorans|salvadoran)\b", "latinxdad", text, flags=re.IGNORECASE)
        ## Replace terms related to Asian with "azn"
        text = re.sub(r"\b(asian\s+americans|asian\s+american|asians|asian|pan\s+asian|chinese|chinese\s+americans|chinese\s+american|phillipines|filipinos|filipino|filipino\s+americans|filipino\s+american|vietnamese|vietnamese\s+americans|vietnamese\s+american|koreans|korean|korean\s+americans|korean\s+american|japanese|japanese\s+americans|japanese\s+american|cambodians|cambodian|cambodian\s+americans|cambodian\s+american|khmer|laos|lao|laotians|laotian|laotian\s+americans|laotian\s+american|hmong|hmong\s+americans|hmong\s+american|thai|thai\s+americans|thai\s+american)\b", "azn", text, flags=re.IGNORECASE)
        
        
        # 6. Extracting year
        match = re.search(r'\d{4}',date)
        if match:
            year = int(match.group())
        else:
            year = None
        return goid, publisher, text, date, year
    else: 
        # 5. Extracting year
        match = re.search(r'\d{4}',date)
        if match:
            year = int(match.group())
        else:
            year = None
        return goid, publisher, text, date, year

In [8]:
# When using multiple processes, important to eventually close them to avoid memory/resource leaks
try:
    # Define a thread Pool to process multiple XML files simultaneously
    # Default set to num_cores - 1, but may change number of processes depending on instance
    p = Pool(processes=num_cores-1)
    
    # Apply function with Pool to corpus
    processed_lists = p.map(make_lists, input_files)

except Exception as e:
    print(f"Error in processing document: {e}")
    
finally:
    p.close()

In [9]:
# Transform processed data into a dataframe
df = pd.DataFrame(processed_lists, columns=['GOID', 'Publisher', 'Text', 'Date', 'Year'])

In [10]:
# Save output to file
df.to_csv(output_file)